In [ ]:
import os
import json
import glob
import random
import collections
import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
rc('animation', html='jshtml')
from tqdm import tqdm

In [ ]:
scans = !ls ../input/siim-acr-pneumothorax-segmentation/stage_2_images -d
print(len(scans))
print(scans[0])

In [ ]:
files=os.listdir('../input/siim-acr-pneumothorax-segmentation/stage_2_images')
print(len(files))
n=len(files)
N=[]
for i in range(n//10):
    N+=[i*10]

In [ ]:
def load_dicom(path):
    dicom=pydicom.read_file(path,force=True)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
def load_dicom_line(path):
    t_paths = sorted(
        glob.glob(os.path.join(path,"*")), 
        key=lambda x: x[:-4].split("_")[-1],
    )
    images = []
    for filename in tqdm(np.array(t_paths)[N]):
        data = load_dicom(filename)
        if data.max() == 0:
            continue
        images.append(data)
        
    return images

In [ ]:
def create_animation(ims):
    fig=plt.figure(figsize=(7,7))
    plt.axis('off')
    im=plt.imshow(ims[0],cmap="gray")

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000)

In [ ]:
images = load_dicom_line(scans[0])

In [ ]:
create_animation(images)